In [ ]:
import os
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import snapatac2 as snap
import scanpy as sc

os.environ[ 'NUMBA_CACHE_DIR' ] = '/tmp/'
sc.settings.verbosity = 3 # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white')

In [ ]:
path = './'

atac_adata_path = os.path.join(path, [f for f in os.listdir(path) if f.endswith('_atac.h5ad')][0])
atac_adata = snap.read(atac_adata_path, backed=None)

peak_path = os.path.join(path, [f for f in os.listdir(path) if f.endswith('peak_mtx.h5ad')][0])
peak_adata = snap.read(peak_path, backed=None)

motif_path = os.path.join(path, [f for f in os.listdir(path) if f.endswith('motif_enrichment.pkl')][0])
with open(motif_path, "rb") as f:
    motifs = pickle.load(f)

rna_adata_path = os.path.join(path, [f for f in os.listdir(path) if f.endswith('_rna.h5ad')][0])
rna_adata = snap.read(rna_adata_path, backed=None)

joint_embedding = os.path.join(path, [f for f in os.listdir(path) if f.endswith('joint_embedding.npy')][0])
joint_embedding = np.load(joint_embedding)

pruned_network_path = os.path.join(path, [f for f in os.listdir(path) if f.endswith('pruned_network.pkl')][0])
with open(pruned_network_path, "rb") as f:
    pruned_network = pickle.load(f)

## Fragment size distribution

In [ ]:
snap.pl.frag_size_distr(atac_adata, interactive=False)

## TSS enrichment

In [ ]:
snap.pl.tsse(atac_adata, interactive=False)

## Clustering

In [ ]:
rna_adata.obs['leiden_atac'] = atac_adata.obs['leiden']
rna_adata.obs['leiden_rna'] = rna_adata.obs['leiden']
atac_adata.obs['leiden_atac'] = atac_adata.obs['leiden']
atac_adata.obs['leiden_rna'] = rna_adata.obs['leiden']
atac_adata.obs['predicted_labels_1'] = rna_adata.obs['predicted_labels_1']

In [ ]:
fig, ax = plt.subplots(1, 2, figsize = (10, 3))
sc.pl.umap(atac_adata, color='leiden_atac', ax = ax[0], show = False, title='ATAC (ATAC leiden)', frameon=False)
sc.pl.umap(rna_adata, color='leiden_atac', ax = ax[1], show = False, title='RNA (ATAC leiden)', frameon=False)
plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize = (10, 3))
sc.pl.umap(atac_adata, color='leiden_rna', ax = ax[0], show = False, title='ATAC (RNA leiden)', frameon=False)
sc.pl.umap(rna_adata, color='leiden_rna', ax = ax[1], show = False, title='RNA (RNA leiden)', frameon=False)
plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize = (10, 3))
sc.pl.umap(atac_adata, color='predicted_labels_1', ax = ax[0], show = False, title='ATAC (RNA celltypist)', frameon=False)
sc.pl.umap(rna_adata, color='predicted_labels_1', ax = ax[1], show = False, title='RNA (RNA celltypist)', frameon=False)
plt.tight_layout()
plt.show()

In [ ]:
if plots is not None: 
    markers = plots.split(",")
    sc.pl.umap(rna_adata, color = markers, frameon = False, sort_order=True)

## Joint Embedding

In [ ]:
# joint embedding
atac_adata.obsm['X_joint'] = joint_embedding
snap.tl.umap(atac_adata, use_rep='X_joint')

In [ ]:
fig, ax = plt.subplots(1, 2, figsize = (10, 3))
sc.pl.umap(atac_adata, color="leiden_atac", title = "Joint embedding (ATAC leiden)", frameon=False, show = False, ax = ax[0])
sc.pl.umap(atac_adata, color="leiden_rna", title = "Joint embedding (RNA leiden)", frameon=False, show = False, ax = ax[1])
plt.tight_layout()
plt.show()

## Marker regions

In [ ]:
marker_peaks = snap.tl.marker_regions(peak_adata, groupby='leiden', pvalue=0.01)
snap.pl.regions(peak_adata, groupby='leiden', peaks=marker_peaks, interactive=False)

## Motif enrichment

In [ ]:
snap.pl.motif_enrichment(motifs, min_log_fc=3, max_fdr=1e-2, height=1000, interactive=False)

## Network Analysis

In [ ]:
snap.pl.network_edge_stat(pruned_network,
                       interactive = False)

In [ ]:
snap.pl.network_scores(pruned_network, score_name = "cor_score", 
                       interactive = False)